# Basketball challenge

In [22]:
from PIL import Image
import torch
import torchvision
from Dataset import Basketball
from Dataprocess import Preprocess
from modules import LinearNet

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Image transformation

In [23]:
img_transformation = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64,48)),
    torchvision.transforms.CenterCrop((48,48)),
    torchvision.transforms.ToTensor()
])


# Preprocessing Dataset

In [24]:
dataprocess = Preprocess().background_subtractor

# Loading Dataset

In [25]:
path = "data"
dataset = Basketball(path, split='training', num_frame = 100, img_transform = img_transformation, dataprocess=dataprocess)

# Display Video

In [ ]:
from IPython import display
import torchvision.transforms.functional as F
view1 = dataset.__getitem__(0)[1]
for index in range(1):
    view1 = dataset.__getitem__(index)[0]
    for idx, img in enumerate(view1):
        img1 = F.to_pil_image(img)
        display.display(img1.resize((320,240)), Image.NEAREST)
        display.clear_output(wait=True)

# Linear Model

In [26]:
variables = dataset[0][0].numel()
model = LinearNet(variables, 2)

In [27]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.4, nesterov=True)
loss_fn = torch.nn.CrossEntropyLoss().cuda()
model = model.cuda()

In [33]:
def train(model, dataset):
    model.train()
    for i in range(0,100):
        samples = dataset[i]
        sample = samples[0].view(1,-1)
        sample = sample.cuda()
        target = torch.tensor([samples[2]])
        target = target.cuda() 

        output = model(sample)
        optimizer.zero_grad()
        loss = loss_fn(output, target)
        loss.backward()
        print(i,' ',loss)   

In [32]:
 def evaluate(model, dataset):
    model.eval()
    eval_loss = 0
    correct = 0
    with torch.no_grad():
        for i in range(100,135):
            samples = dataset[i]
            sample = samples[0].view(1,-1)
            sample = sample.cuda()
            target = torch.tensor([samples[2]])
            target = target.cuda() 
            output = model(sample)

            eval_loss += loss_fn(output, target)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    eval_loss /= 35
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        eval_loss, correct, 35,
        100. * correct / 35))

In [34]:
epochs = 10
for epoch in range(1, epochs+1):
    train(model, dataset)
    evaluate(model, dataset)

0   tensor(0.6778, device='cuda:0', grad_fn=<NllLossBackward>)
1   tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward>)
2   tensor(0.7014, device='cuda:0', grad_fn=<NllLossBackward>)
3   tensor(0.6741, device='cuda:0', grad_fn=<NllLossBackward>)
4   tensor(0.6947, device='cuda:0', grad_fn=<NllLossBackward>)
5   tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward>)
6   tensor(0.6961, device='cuda:0', grad_fn=<NllLossBackward>)
7   tensor(0.7196, device='cuda:0', grad_fn=<NllLossBackward>)
8   tensor(0.7269, device='cuda:0', grad_fn=<NllLossBackward>)
9   tensor(0.6710, device='cuda:0', grad_fn=<NllLossBackward>)
10   tensor(0.7147, device='cuda:0', grad_fn=<NllLossBackward>)
11   tensor(0.7128, device='cuda:0', grad_fn=<NllLossBackward>)
12   tensor(0.6869, device='cuda:0', grad_fn=<NllLossBackward>)
13   tensor(0.6894, device='cuda:0', grad_fn=<NllLossBackward>)
14   tensor(0.7029, device='cuda:0', grad_fn=<NllLossBackward>)
15   tensor(0.6937, device='cuda:0', grad_fn=<NllL

NameError: name 'test' is not defined